In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, timedelta
from pathlib import Path

import copy
import pyarrow.parquet as pq

In [2]:
data_path = Path('../data')
data_path.exists()

True

In [ ]:
df_train = pd.read_csv('../data/train.csv')

<h3> concatenate book data for train and test</h3>

In [ ]:
src_folder = 'z_1'

In [ ]:
stock_list = df_train['stock_id'].unique().tolist()
print(len(stock_list), stock_list[:2])

In [ ]:
corr_list = []

for stock_id in stock_list:
    df_target = df_train[df_train['stock_id']==stock_id].copy(deep=True)
    file_loc = data_path.joinpath(f'{src_folder}/{stock_id}')
    df = pd.read_csv(file_loc, sep='|', compression='bz2', index_col=0)
    
    df=df.merge(df_target, on=['stock_id', 'time_id'], how='inner')
    df['target_log']=np.log(df['target'])
    df_corr=df.corr()[['target', 'target_log']].abs()
    df_corr.drop(index=['stock_id', 'target', 'time_id', 'target_log'], inplace=True)
    df_corr.columns=[f'stock={stock_id}', f'stock_log={stock_id}']
    
    corr_list.append(df_corr)

In [ ]:
df_all = pd.concat(corr_list, axis=1, join='inner') 
df_all.index.name='feature'

In [ ]:
df_all.abs().to_excel('../data/y_1/feature_importance_book.xlsx', index=True)

### trade data features

In [ ]:
src_folder ='z_2' #'z_1'

In [ ]:
stock_list = df_train['stock_id'].unique().tolist()
print(len(stock_list), stock_list[:2])

In [ ]:
corr_list = []

for stock_id in stock_list:
    df_target = df_train[df_train['stock_id']==stock_id].copy(deep=True)
    file_loc = data_path.joinpath(f'{src_folder}/{stock_id}')
    df = pd.read_csv(file_loc, sep='|', compression='bz2', index_col=0)
    
    df=df.merge(df_target, on=['stock_id', 'time_id'], how='inner')
    df['target_log']=np.log(df['target'])
    df_corr=df.corr()[['target', 'target_log']].abs()
    df_corr.drop(index=['stock_id', 'target', 'time_id', 'target_log'], inplace=True)
    df_corr.columns=[f'stock={stock_id}', f'stock_log={stock_id}']
    
    corr_list.append(df_corr)

In [ ]:
df_all = pd.concat(corr_list, axis=1, join='inner') 
df_all.index.name='feature'

In [ ]:
df_all.abs().to_excel('../data/y_1/feature_importance_trade.xlsx', index=True)

## combine

In [10]:
df_1=pd.read_excel('../data/y_1/feature_importance_book.xlsx', engine='openpyxl', index_col=0).abs()
df_2=pd.read_excel('../data/y_1/feature_importance_trade.xlsx', engine='openpyxl', index_col=0).abs()

In [11]:
df_1['folder']='z_1'
df_2['folder']='z_2'

In [12]:
df_1.shape, df_2.shape

((808, 225), (818, 225))

In [13]:
set(df_1.columns.tolist())-set(df_2.columns.tolist())

set()

In [14]:
df_1[df_2.columns.tolist()].append(df_2).to_excel('../data/y_1/feature_importance_all.xlsx', index=True)